In [1]:
import tensorflow as tf
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
from keras.backend.tensorflow_backend import set_session
config = tf.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.3
set_session(tf.Session(config=config))

%matplotlib inline

Using TensorFlow backend.


In [2]:
%run 'CNNBlocks.ipynb'
%run 'modelArchitecture.ipynb'
%run 'batchGenerator.ipynb'

In [3]:
model = compileModel((1,64,64,64),1e-4)

/opt/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:5: UserWarning: Update your `Conv3D` call to the Keras 2 API: `Conv3D(8, (3, 3, 3), data_format="channels_first", padding="same", kernel_regularizer=<keras.reg...)`
/opt/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:12: UserWarning: Update your `MaxPooling3D` call to the Keras 2 API: `MaxPooling3D(data_format="channels_first")`
/opt/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:6: UserWarning: Update your `AveragePooling3D` call to the Keras 2 API: `AveragePooling3D(data_format="channels_first")`
/opt/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:7: UserWarning: The `merge` function is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
/opt/anaconda/lib/python3.5/site-packages/keras/legacy/layers.py:456: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras

Compiling model...


/opt/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:10: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(1, activation="sigmoid", kernel_regularizer=<keras.reg..., name="Spiculation")`
/opt/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:33: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=[<tf.Tenso...)`


In [ ]:
# model_64.load_weights('/home/katya/LungCancer/Katya/CNN_v1/model_v1_weights_temp64.h5')

modelPath = '/home/katya/LungCancer/Katya/CNN_v2/model_and_weights/LUNA_model_v2.h5'
model, lossHist = trainModel(model, modelPath, batchSize=30, nbEpoch = 30, stepsPerEpoch = 150)

model.save_weights('/home/katya/LungCancer/Katya/CNN_v2/model_and_weights/LUNA_model_v2_weights.h5')

Loading positive patches
Loading negative patches
Loading false positive patches


/opt/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:43: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<generator..., epochs=1, validation_data=<generator..., initial_epoch=0, callbacks=[<keras.ca..., steps_per_epoch=150, validation_steps=1)`


Epoch 1/1
150/150 [==============================] - 299s - loss: 0.7456 - Malignancy_loss: 0.0904 - Diameter_loss: 0.0319 - Lobulation_loss: 0.0429 - Spiculation_loss: 0.0449 - val_loss: 0.8290 - val_Malignancy_loss: 0.1110 - val_Diameter_loss: 0.0256 - val_Lobulation_loss: 0.0511 - val_Spiculation_loss: 0.0390

/opt/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:43: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<generator..., epochs=2, validation_data=<generator..., initial_epoch=1, callbacks=[<keras.ca..., steps_per_epoch=150, validation_steps=1)`


150/150 [==============================] - 297s - loss: 0.6790 - Malignancy_loss: 0.0855 - Diameter_loss: 0.0265 - Lobulation_loss: 0.0382 - Spiculation_loss: 0.0370 - val_loss: 1.2043 - val_Malignancy_loss: 0.1724 - val_Diameter_loss: 0.0461 - val_Lobulation_loss: 0.0672 - val_Spiculation_loss: 0.0519

/opt/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:43: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<generator..., epochs=3, validation_data=<generator..., initial_epoch=2, callbacks=[<keras.ca..., steps_per_epoch=150, validation_steps=1)`


 86/150 [================>.............] - ETA: 126s - loss: 0.6554 - Malignancy_loss: 0.0816 - Diameter_loss: 0.0275 - Lobulation_loss: 0.0364 - Spiculation_loss: 0.0376

In [ ]:
plt.figure(figsize=[10,7])
plt.grid(True, ls='--', lw=0.5, alpha=0.5, dash_capstyle = 'round', c='gray')
plt.xlabel('Epoch #', fontsize=15)
plt.ylabel('Loss', fontsize=15)

labels=list(lossHist.keys())

for i,key in enumerate(lossHist):
    plt.plot([x for x in range(30)], lossHist[key], 'o', label=labels[i])
    plt.legend()
    
plt.savefig('/home/katya/LungCancer/Katya/CNN_v2/figures/3.png')